In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_datasets as tfds
import tf_keras as keras

from keras import utils
from tensorflow_examples.models.pix2pix import pix2pix

dataset = tfds.load('oxford_iiit_pet')

In [ ]:
!pip install -q git+https://github.com/tensorflow/examples.git

In [ ]:
IMG_SIZE = (128, 128)
EPOCHS = 50

TRAIN_LENGTH = len(dataset['train'])
BATCH_SIZE = 64
BUFFER_SIZE = 1000
STEPS_PER_EPOCH = TRAIN_LENGTH // BATCH_SIZE

При анализе данных было выявлено, что размер изображений разный. 
Многие модели требуют одинаковый размер входных данных, чтобы модель могла правильно их обрабатывать.

# Разделение датасета на тренировочный и тестовый. Предобработка данных.

1. Нормализация. Приводим значения пикселей изображений с диапазона [0; 255] к диапазону [0; 1]
2. Приводим изображения и маски к одному размеру.

Нормализация данных гаранитрует, что модель получает однотипные входные данные, что упрощает обучение и повышает ее точность.
Приведение данных в разных единицах измерения и диапазонах значений к единому виду позволяет сравнивать их между собой или использовать для расчёта схожести объектов. 
Модель эффективнее извлекает признаки и быстрее обучается.

In [ ]:
def resize_normalize(input_image, input_mask):
    input_image = tf.image.resize(input_image, IMG_SIZE)
    input_mask = tf.image.resize(input_mask, IMG_SIZE)
    input_image = tf.cast(input_image, tf.float32) / 255.0
    input_mask -= 1
    return input_image, input_mask

def load_image_train(example):
    return resize_normalize(example['image'], example['segmentation_mask'])

def load_image_test(example):
    return resize_normalize(example['image'], example['segmentation_mask'])

In [ ]:
train_dataset = dataset['train'].map(load_image_train, num_parallel_calls=tf.data.AUTOTUNE)
test_dataset = dataset['test'].map(load_image_test)

train_data = train_dataset.cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE).repeat()
train_data = train_data.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
test_data = test_dataset.batch(BATCH_SIZE)

In [ ]:
def display(display_list):
  plt.figure(figsize=(15,15))

  title = ['Input Image', 'True Mask', 'Predicted Mask']

  for i in range(len(display_list)):
    plt.subplot(1, len(display_list), i+1)
    plt.title(title[i])
    plt.imshow(tf.keras.preprocessing.image.array_to_img(display_list[i]))
    plt.axis('off')
  plt.show()

In [ ]:
for image, mask in train_dataset.take(10):
  sample_image, sample_mask = image, mask
display([sample_image, sample_mask])
for image, mask in train_dataset.take(20):
  sample_image, sample_mask = image, mask
display([sample_image, sample_mask])

Выше представлены изображения и маски после нормализации. Размер изображений приведен к одному.

# Обучение модели

Для обучения была выбрана модель MobileNetV2. Улучшенная версия оригинальной MobileNet и разработанная специально для задач CV. Её основной компонет - глубокая свертка, которая позволяет снижать параметры и при этом сохранять хорошее качество. Также она оперирует более широкими слоями, что улучшает качество предсказаний за счет повышенной сложности.


- `include_top=False` задает, что верхние полносвязные слои не будут включены в загруженную модель (не будут использоваться).
- В `names` задаются названия слоев, из которых будет состоять подграфик модели.
- В `up_stack` определяются последовательные слои для апсемплинга. Каждый `pix2pix.upsample` слой увеличивает размер изображения в 2 раза с использованием ядра свертки размером 3x3.

- Для увеличения размера изображения до начального признакового пространства используется слой `tf.keras.layers.Conv2DTranspose`.

- Модель компилируется с оптимизатором Adam, функцией потерь SparseCategoricalCrossentropy, и метрикой accuracy для оценки производительности модели.



In [ ]:
base_model = tf.keras.applications.MobileNetV2(input_shape=(128,128,3), include_top=False)

names = ['block_1_expand_relu', 'block_3_expand_relu', 'block_6_expand_relu',
         'block_13_expand_relu', 'block_16_expand_relu']
layers = [base_model.get_layer(name).output for name in names]

down_sample = tf.keras.Model(base_model.input, layers)
down_sample.trainable = False

up_stack = [
    pix2pix.upsample(512, 3),
    pix2pix.upsample(256, 3),  
    pix2pix.upsample(128, 3),  
    pix2pix.upsample(64, 3),   
]

inputs = tf.keras.layers.Input(shape=[128,128,3 ])
x = inputs

skips = down_sample(x)
x = skips[-1]
skips = reversed(skips[:-1])

for up, skip in zip(up_stack, skips):
  x = up(x)
  concat = tf.keras.layers.Concatenate()
  x = concat([x, skip])

x = tf.keras.layers.Conv2DTranspose(3, 3, strides=2 , padding='same')(x)

model = tf.keras.Model(inputs, x)

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy']
             )

In [ ]:
def create_mask(pred_mask):
  pred_mask = tf.argmax(pred_mask, axis=-1)
  pred_mask = pred_mask[..., tf.newaxis]
  return pred_mask[0]

def show_predictions(dataset=None, num=1):
    if dataset:
        for image, mask in dataset.take(num):
          pred_mask = model.predict(image)
          display([image[0], mask[0], create_mask(pred_mask)])

In [ ]:
model.fit(train_data, 
          epochs=EPOCHS,
          steps_per_epoch=STEPS_PER_EPOCH,
          validation_data=test_data,
     )

В качестве метрики для оценки качества обучения была выбрана "accuracy", которая характеризует качество модели, агрегированное по всем классам. Она представляет собой простую и понятную метрику. Хорошо подходит для задач классификации, где необходимо знать долю правильно угаданных классов.

В результате оценки качества модели на тестовых данных модель смогла достичь точности 90%. 
Для улучшения качества модели можно по-другому настроить значения гиперпараметров, увеличить количество эпох для обучения, также использовать другие методы аугментации для обучения модели на разнообразных данных.

# Предсказанные значения сегментации:

In [ ]:
show_predictions(test_data, 6)